In [ ]:
import tifffile
import numpy as np
import torch
import os
import random
import h5py
import cv2
import math
from scipy.ndimage.interpolation import zoom
from PIL import Image

from tqdm import tqdm
import matplotlib.pyplot as plt

from sr_3dunet.utils.data_utils import random_crop_3d, random_crop_2d, augment_3d, augment_2d, preprocess, get_projection, get_rotated_img, crop_block

In [ ]:
ims_path = '/share/home/wangwb/workspace/sr_3dunet/datasets/cellbody/out_h5/output_res0.h5'
out_ims_path = '/share/home/wangwb/workspace/sr_3dunet/datasets/cellbody/out_h5/output_res2.h5'

h5 = h5py.File(ims_path, 'r')
img_total = h5['DataSet']['ResolutionLevel 0']['TimePoint 0']['Channel 0']['Data']
h5_dir = 'DataSet/ResolutionLevel 2/TimePoint 0/Channel 0/Data'

In [ ]:
step_size = 512

downscale_factor = 4

with h5py.File(out_ims_path, 'w') as f:
    f.create_dataset(h5_dir, shape=img_total.shape, chunks=(1, 128, 128), dtype=img_total.dtype)

len1 = math.ceil(img_total.shape[0]/step_size)
len2 = math.ceil(img_total.shape[1]/step_size)
len3 = math.ceil(img_total.shape[2]/step_size)
pbar1 = tqdm(total=len1*len2*len3, unit='h5_img', desc='inference')

for start_h in range(0, img_total.shape[0], step_size):
    end_h = img_total.shape[0] if start_h+step_size>img_total.shape[0] else start_h+step_size
    for start_w in range(0, img_total.shape[1], step_size):
        end_w = img_total.shape[1] if start_w+step_size>img_total.shape[1] else start_w+step_size
        for start_d in range(0, img_total.shape[2], step_size):
            end_d = img_total.shape[2] if start_d+step_size>img_total.shape[2] else start_d+step_size
            
            new_shape = ((end_h-start_h) // downscale_factor,
             (end_w-start_w) // downscale_factor,
             (end_d-start_d) // downscale_factor)
            
            # img_total[start_h:end_h, start_w:end_w, start_d:end_d].resize(new_shape, resample=Image.BICUBIC)
            new_img = zoom(img_total[start_h:end_h, start_w:end_w, start_d:end_d], zoom = 0.5, order=1)

            with h5py.File(out_ims_path, 'r+') as f:
                f[h5_dir][start_h//2:end_h//2, start_w//2:end_w//2, start_d//2:end_d//2] = new_img

            pbar1.update(1)

In [ ]:
original_matrix = np.random.rand(1000, 1000, 1000)



new_shape = (original_matrix.shape[0] // downscale_factor,
             original_matrix.shape[1] // downscale_factor,
             original_matrix.shape[2] // downscale_factor)

new_matrix = np.zeros(new_shape)
for i in range(new_shape[0]):
    for j in range(new_shape[1]):
        for k in range(new_shape[2]):
            new_matrix[i, j, k] = np.mean(original_matrix[i*downscale_factor:(i+1)*downscale_factor,
                                                          j*downscale_factor:(j+1)*downscale_factor,
                                                          k*downscale_factor:(k+1)*downscale_factor])
